In [1]:
%%javascript
function ClickConnect(){
    console.log("Working");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
dataset_path = "/content/drive/MyDrive/Colab Notebooks/Sunidhi/DATASET"  # Update this path
train_dir = os.path.join(dataset_path, "TRAIN")
test_dir = os.path.join(dataset_path, "TEST")
model_path = "/content/drive/MyDrive/Colab Notebooks/Sunidhi/models/waste_classification_mobilnet.h5"

# Image dimensions and batch size
img_size = (224, 224)
batch_size = 32

In [4]:
# Check if the model already exists
if os.path.exists(model_path):
    # Load the pre-trained model
    print("Loading pre-trained model...")
    model = tf.keras.models.load_model(model_path)
else:
    # Data augmentation and preprocessing
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255.0,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.2  # 80% training, 20% validation
    )

    test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

    # Load training and validation data
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode="binary",
        subset="training"
    )

    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode="binary",
        subset="validation"
    )

    # Load test data
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode="binary"
    )

    # Load a pre-trained model (MobileNetV2) for transfer learning
    base_model = MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=(224, 224, 3)
    )

    # Freeze the base model layers
    base_model.trainable = False

    # Add custom layers on top of the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation="sigmoid")(x)  # Binary classification

    # Create the final model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    # Callbacks
    checkpoint = ModelCheckpoint(
        model_path,
        monitor="val_accuracy",
        save_best_only=True,
        mode="max"
    )

    early_stopping = EarlyStopping(
        monitor="val_accuracy",
        patience=5,
        restore_best_weights=True
    )

    # Train the model
    print("Training the model...")
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
        epochs=20,
        callbacks=[checkpoint, early_stopping]
    )

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

    # Save the model
    model.save(model_path)
    print("Model saved.")

Loading pre-trained model...


In [5]:
# Function to test the model on a new image
def test_new_image(img_path):
    # Load and preprocess the new image
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    # Make a prediction
    prediction = model.predict(img_array)
    predicted_class = "Organic" if prediction[0] < 0.5 else "Recyclable"
    print(f"Predicted Class: {predicted_class}")

In [6]:
# Test the model on a new image
new_image_path = "/content/drive/MyDrive/Colab Notebooks/Sunidhi/battery.jpg"
test_new_image(new_image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Predicted Class: Recyclable
